In [1]:
import pickle
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from seq2seq import *
import pandas as pd
import numpy as np
import torch
import random
import itertools
from sklearn.preprocessing import StandardScaler

In [2]:
def set_seeds(seed):
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
# Convert a string that simulates a list to a real list
def convert_string_list(element):
    # Delete [] of the string
    element = element[0:len(element)]
    # Create a list that contains each code as e.g. 'A'
    ATC_list = list(element.split('; '))
    for index, code in enumerate(ATC_list):
        # Delete '' of the code
        ATC_list[index] = code[0:len(code)]
    return ATC_list

In [4]:
train_set = pd.read_csv('../../Data/train_set.csv')
test_set = pd.read_csv('../../Data/test_set.csv')
val_set = pd.read_csv('../../Data/val_set.csv')
set_seeds(78)

In [5]:
def multiplicate_rows(df):
    # Duplicate each compound the number of ATC codes associated to it, copying its SMILES in new rows
    new_rows = []
    
    for _, row in df.iterrows():
        atc_codes = row['ATC Codes']
        atc_codes_list = convert_string_list(atc_codes)
        
        if len(atc_codes_list) > 1:
            for code in atc_codes_list:
                if len(code) == 5:
                    new_row = row.copy()
                    new_row['ATC Codes'] = code
                    new_rows.append(new_row)
        else:
            if len(atc_codes_list[0]) == 5:
                new_rows.append(row)
    
    new_set = pd.DataFrame(new_rows)
    new_set = new_set.reset_index(drop=True)

    return new_set
    
# Create vocabularies
# Tokenize the data
def source(df):
    source = []
    for compound in df['Neutralized SMILES']:
        # A list containing each SMILES character separated
        source.append(list(compound))
    return source
def target(df):
    target = []
    for codes in df['ATC Codes']:  
        code = convert_string_list(codes) 
        # A list of lists, each one containing each ATC code character separated 
        for c in code:
            list_c = list(c)
            target.append(list_c)
    return target

In [6]:
new_train_set = multiplicate_rows(train_set)
new_val_set = multiplicate_rows(val_set)
new_test_set = multiplicate_rows(test_set)

new_test_set.to_csv("onecodeperdrug_test_set.csv", index = False)
new_val_set.to_csv("onecodeperdrug_val_set.csv", index = False)

source_train = source(new_train_set)
source_test = source(new_test_set)
# Test set without duplicated compounds
source_test2 = source(test_set)
source_val = source(new_val_set)
# Val set without duplicated compounds
source_val2 = source(val_set)

target_train = target(new_train_set)
target_test = target(new_test_set)
target_val = target(new_val_set)

# An Index object represents a mapping from the vocabulary to integers (indices) to feed into the models
source_index = index.Index(source_train)
target_index = index.Index(target_train)

# Create tensors
X_train = source_index.text2tensor(source_train)
y_train = target_index.text2tensor(target_train)
X_val = source_index.text2tensor(source_val)
X_val2 = source_index.text2tensor(source_val2)
y_val = target_index.text2tensor(target_val)     
X_test = source_index.text2tensor(source_test)
X_test2 = source_index.text2tensor(source_test2)
y_test = target_index.text2tensor(target_test)

if torch.cuda.is_available():
    X_train = X_train.to("cuda")
    y_train = y_train.to("cuda")
    X_val = X_val.to("cuda")
    X_val2 = X_val2.to("cuda")
    y_val = y_val.to("cuda")
    X_test= X_test.to("cuda")
    y_test = y_test.to("cuda")
    X_test2 = X_test2.to("cuda")

In [7]:
hyperparameters_grid = { 
    'enc_embedding_dim': [32, 64, 128],
    'dec_embedding_dim': [32, 64, 128],
    'enc_hidden_units': [32, 64, 128],
    'dec_hidden_units': [32, 64, 128],
    'enc_layers': [2, 3, 4],
    'dec_layers': [2, 3, 4],
    'dropout': [0.0, 0.1, 0.2],
    'weight_decays': [10**-4, 10**-5],
    'learning_rates': [10**-3, 10**-4]
}


In [8]:
# Randomly sample from dictionary
random_params = {k: random.sample(v, 1)[0] for k, v in hyperparameters_grid.items()}
print(random_params['enc_embedding_dim'])

128


In [9]:
def random_search(max_evals):
    tested_params = set()
    df_tests = pd.DataFrame(columns = ['#epochs', 'encoder_embedding_dim', 'decoder_embedding_dim', 'enc_layers', 'dec_layers', 'enc_hidden_units', 'dec_hidden_units', 'dropout', 'weight_decay', 'learning_rate', 'Precisionatn nivel1', 'Precisionatn nivel2', 'Precisionatn nivel3', 'Precisionatn nivel4', 'Precision nivel1', 'Precision nivel2', 'Precision nivel3', 'Precision nivel4', 'Recall nivel1', 'Recall nivel2', 'Recall nivel3', 'Recall nivel4', 'Drugs that have at least one match'], index = list(range(max_evals)))
    sys.stdout = open('log.txt', 'w')
    for i in range(max_evals):
        while True:
            random_params = {k: random.sample(v, 1)[0] for k, v in hyperparameters_grid.items()}
            params_tuple = tuple(random_params.values())
            if params_tuple not in tested_params:
                tested_params.add(params_tuple)
                break   
        model = models.BiLSTM(
                 source_index, 
                 target_index,
                 encoder_embedding_dimension = random_params['enc_embedding_dim'],
                 decoder_embedding_dimension = random_params['dec_embedding_dim'],
                 encoder_hidden_units = random_params['enc_hidden_units'],
                 encoder_layers = random_params['enc_layers'],
                 decoder_hidden_units = random_params['dec_hidden_units'],
                 decoder_layers = random_params['dec_layers'],
                 dropout = random_params['dropout'])   
        model.to("cuda")
        model.fit(
                X_train,
                y_train,
                X_val, 
                y_val, 
                batch_size = 32, 
                epochs = 500, 
                learning_rate = random_params['learning_rates'], 
                weight_decay = random_params['weight_decays'],
                progress_bar = 0, 
                save_path = None
        ) 
        model.load_state_dict(torch.load("best_model.pth", weights_only=True))
        ep = model.early_stopping.best_epoch
        loss, error_rate = model.evaluate(X_val, y_val)    
        predictions, log_probabilities = search_algorithms.beam_search(
            model, 
            X_val,
            predictions = 6, # max length of the predicted sequence
            beam_width = 3,
            batch_size = 32, 
            progress_bar = 0
        )
        output_beam = [target_index.tensor2text(p) for p in predictions]
        predictions2, log_probabilities2 = search_algorithms.beam_search(
            model, 
            X_val2,
            predictions = 6, # max length of the predicted sequence
            beam_width = 3,
            batch_size = 32, 
            progress_bar = 0
        )
        output_beam2 = [target_index.tensor2text(p) for p in predictions2]
        
        predictions_onecodeperdrug = []
        for preds in output_beam:
            interm = []
            for pred in preds:
                clean_pred = pred.replace('<START>', '').replace('<END>', '')
                if len(clean_pred) == 5:
                    interm.append(clean_pred)
            predictions_onecodeperdrug.append(interm)
                
        predictions = []
        for preds in output_beam2:
            interm = []
            for pred in preds:
                clean_pred = pred.replace('<START>', '').replace('<END>', '')
                if len(clean_pred) == 5:
                    interm.append(clean_pred)
            predictions.append(interm)
                
        precisionatn_1, precisionatn_2, precisionatn_3, precisionatn_4 = defined_metrics.precisionatn(predictions_onecodeperdrug, "onecodeperdrug_val_set.csv", 'ATC Codes')
        precision_1, precision_2, precision_3, precision_4 = defined_metrics.precision(predictions, "../../Data/val_set.csv", 'ATC Codes')
        recall_1, recall_2, recall_3, recall_4, comp = defined_metrics.recall(predictions, "../../Data/val_set.csv", 'ATC Codes')
        df_tests.iloc[i, :] = [f"{ep}", f"{random_params['enc_embedding_dim']}", f"{random_params['dec_embedding_dim']}", f"{random_params['enc_layers']}", f"{random_params['dec_layers']}", f"{random_params['enc_hidden_units']}", f"{random_params['dec_hidden_units']}", f"{random_params['dropout']}", f"{random_params['weight_decays']}", f"{random_params['learning_rates']}", f"{precisionatn_1}", f"{precisionatn_2}", f"{precisionatn_3}", f"{precisionatn_4}", f"{precision_1}", f"{precision_2}", f"{precision_3}", f"{precision_4}", f"{recall_1}", f"{recall_2}", f"{recall_3}", f"{recall_4}", f"{comp}"]
        df_tests.to_csv("bilstm_results.csv", index = False)
    sys.stdout = sys.__stdout__
    return df_tests

In [10]:
df_tests = random_search(200)

In [11]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import pandas as pd
# from matplotlib.lines import Line2D

# # Cargar datos
# df_res = pd.read_csv("results_transformer.csv")
# df_res = df_res[["embedding_dim", "enc_layers", "dec_layers", "attention_heads", "dropout", 
#                  "weight_decay", "learning_rate", "Precision nivel1", "Recall nivel1"]]

# size_map = {32: 100, 64: 200, 128: 300}
# df_res['embedding dimension'] = df_res['embedding_dim'].map(size_map)
# # Normalizar tamaño de los puntos
# # df_res['embedding dimension'] = (df_res['embedding_dim'] - df_res['embedding_dim'].min()) / (df_res['embedding_dim'].max() - df_res['embedding_dim'].min()) * 100 + 50

# df_res['encoder and decoder layers'] = df_res.apply(
#     lambda row: f'enc_{int(row["enc_layers"])} - dec_{int(row["dec_layers"])}', axis=1)
# df_res['weight_decay and learning_rate'] = df_res.apply(
#     lambda row: f'WD_{row["weight_decay"]} - LR_{row["learning_rate"]}', axis=1)

# # Mapear 'dropout' a formas
# dropout_map = {0.0: 'o', 0.1: '^', 0.2: 's'}
# df_res['dropout value'] = df_res['dropout'].map(dropout_map)

# df_res['edgewidth'] = df_res['attention_heads'].map({2: 1.0, 4: 2.0})
# color_palette = sns.color_palette("Pastel1", n_colors=df_res['encoder and decoder layers'].nunique())

# # Mapa de colores para 'weight_decay and learning_rate'
# unique_wd_lr = df_res['weight_decay and learning_rate'].unique()
# color_palette2 = sns.color_palette("Set1", n_colors=len(unique_wd_lr))
# wd_lr_color_map = dict(zip(unique_wd_lr, color_palette2))

# # Mapa de colores para 'encoder and decoder layers'
# unique_enc_dec = df_res['encoder and decoder layers'].unique()
# enc_dec_color_map = dict(zip(unique_enc_dec, color_palette))

# plt.figure(figsize=(12, 12))

# # Dibujar puntos con Matplotlib
# for i, row in df_res.iterrows():
#     plt.scatter(
#         x=row['Precision nivel1'],
#         y=row['Recall nivel1'],
#         s=row['embedding dimension'],  # Tamaño de los puntos
#         color=enc_dec_color_map[row['encoder and decoder layers']],  # Color del punto
#         marker=row['dropout value'],  # Forma del punto
#         edgecolor=wd_lr_color_map[row['weight_decay and learning_rate']],  # Borde basado en mapa precalculado
#         linewidth=row['edgewidth'],  # Grosor del borde
#         alpha=0.7
#     )
    
# legend_elements = [
#     Line2D([0], [0], color='none', label='Dropout'),
#     Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=10, label='Dropout 0.0'),
#     Line2D([0], [0], marker='^', color='w', markerfacecolor='black', markersize=10, label='Dropout 0.1'),
#     Line2D([0], [0], marker='s', color='w', markerfacecolor='black', markersize=10, label='Dropout 0.2'),
#     Line2D([0], [0], color='none', label='Attention heads'),
#     Line2D([0], [0], color='black', linewidth=1.0, label='2 attention heads'),
#     Line2D([0], [0], color='black', linewidth=2.0, label='4 attention heads'),
#     Line2D([0], [0], color='none', label='Encoder - Decoder layers'),
#     *[Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label) for label, color in enc_dec_color_map.items()],
#     Line2D([0], [0], color='none', label='Weight decay - Learning rate'),
#     *[Line2D([0], [0], marker='o', color='w', markeredgecolor=color, markerfacecolor='none', markersize=10, label=label) for label, color in wd_lr_color_map.items()]
# ]

# size_mapping = {
#     100: "32",
#     200: "64",
#     300: "128"
# }

# size_legend = [
#     Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=size/18, label=f'{size_mapping.get(int(size))}') 
#     for size in df_res['embedding dimension'].unique() 
# ]

# plt.legend(title='Hiperparámetros', handles=[Line2D([0], [0], color='none', label='Embedding Dimension')] + size_legend + legend_elements, loc='upper left', bbox_to_anchor=(1.05, 1))

# plt.title("Combinación de hiperparámetros", fontsize=16)
# plt.xlabel("Precision nivel 1", fontsize=14)
# plt.ylabel("Recall nivel 1", fontsize=14)
# plt.tight_layout()
# plt.savefig('multimodaltransformer_hyperparams.png', bbox_inches='tight')
# plt.show()


In [12]:
df_tests.sort_values(by = "Precision nivel1")

,#epochs,encoder_embedding_dim,decoder_embedding_dim,enc_layers,dec_layers,enc_hidden_units,dec_hidden_units,dropout,weight_decay,learning_rate,...,Precisionatn nivel4,Precision nivel1,Precision nivel2,Precision nivel3,Precision nivel4,Recall nivel1,Recall nivel2,Recall nivel3,Recall nivel4,Drugs that have at least one match
59,294,64,32,4,4,32,64,0.0,0.0001,0.0001,...,84.21052631578947,0.1309216192937123,0.2727272727272727,0.6333333333333333,0.6052631578947368,0.26546080964685614,0.2681818181818182,0.6166666666666667,0.8421052631578947,"[110, 30, 19, 16]"
14,269,32,128,3,4,32,32,0.1,0.0001,0.0001,...,20.0,0.14211886304909555,0.2196969696969697,0.4,0.15,0.26485788113695086,0.22272727272727272,0.4,0.2,"[110, 25, 10, 2]"
193,228,64,128,2,4,32,32,0.0,0.0001,0.0001,...,50.0,0.14728682170542637,0.20216049382716048,0.5466666666666667,0.4642857142857143,0.26076658053402235,0.21862139917695472,0.54,0.5,"[108, 25, 14, 7]"
114,133,32,64,2,4,32,128,0.0,1e-05,0.0001,...,57.57575757575758,0.15073212747631362,0.37926509186351703,0.62,0.4494949494949495,0.31201550387596894,0.378827646544182,0.65,0.5757575757575758,"[127, 50, 33, 19]"
163,239,64,128,2,3,32,32,0.2,1e-05,0.0001,...,64.28571428571429,0.1515934539190354,0.3364485981308411,0.6495726495726496,0.5,0.2592592592592593,0.3561786085150571,0.6923076923076923,0.6428571428571429,"[107, 39, 28, 18]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,45,128,32,2,2,128,128,0.1,0.0001,0.001,...,78.19905213270142,0.4651162790697675,0.6464435146443513,0.7855902777777778,0.6861598440545809,0.5929514786103933,0.775092980009298,0.8472470238095237,0.8115009746588693,"[239, 192, 171, 144]"
99,28,64,64,4,2,128,128,0.2,1e-05,0.001,...,83.33333333333334,0.47200689061154183,0.7009222661396575,0.7629107981220657,0.7138888888888889,0.627878265862762,0.8208058849363199,0.8108540129666889,0.8537037037037039,"[253, 213, 180, 159]"
144,39,128,32,2,2,128,64,0.1,1e-05,0.001,...,76.036866359447,0.4737295434969856,0.7319727891156462,0.7315270935960592,0.6505681818181818,0.5989376973873098,0.8186281179138323,0.8371979826413324,0.7931818181818181,"[245, 203, 176, 147]"
101,40,128,128,3,2,128,32,0.1,1e-05,0.001,...,67.8048780487805,0.4909560723514214,0.640456989247312,0.782986111111111,0.5994152046783625,0.5972437553832902,0.7477598566308243,0.8566592261904762,0.713840155945419,"[248, 192, 171, 129]"


In [13]:
(df_tests.sort_values(by = "Precision nivel1")).to_csv("bilstm_sortedresults.csv", index = False)